In [2]:
# Append web folder to system path
import sys
sys.path.append('../web')

In [8]:
from db_config import Base
from Database import Station, Availability, Weather
from sqlalchemy import create_engine, func, Column, String, Integer, Double, Boolean
from sqlalchemy.orm import sessionmaker
import json
import sys
import requests

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pickle
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split

In [6]:
# Get the db_info
with open('../dbinfo.json') as f:
    db_info = json.load(f)


USER = db_info['dbConnection']['USER']
PASSWORD = db_info['dbConnection']['PASSWORD']
URI = db_info['dbConnection']['URI']
PORT = db_info['dbConnection']['PORT']
DB = db_info['dbConnection']['DB']
WEATHER_API_KEY = db_info['weatherKey']


# Create a new session
engine = create_engine(
    'mysql+pymysql://{}:{}@localhost:{}/{}'.format(USER, PASSWORD, PORT, DB), echo=True)
Base.metadata.create_all(bind=engine)
Session = sessionmaker(bind=engine)
session = Session()
print("connected")

2024-03-29 15:52:09,825 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2024-03-29 15:52:09,851 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-03-29 15:52:09,917 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2024-03-29 15:52:09,918 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-03-29 15:52:09,933 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2024-03-29 15:52:09,934 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-03-29 15:52:09,961 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-29 15:52:09,962 INFO sqlalchemy.engine.Engine DESCRIBE `db_on_your_bike`.`stations`
2024-03-29 15:52:09,962 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-03-29 15:52:09,979 INFO sqlalchemy.engine.Engine DESCRIBE `db_on_your_bike`.`availability`
2024-03-29 15:52:09,981 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-03-29 15:52:09,998 INFO sqlalchemy.engine.Engine DESCRIBE `db_on_your_bike`.`weather`
2024-03-29 15:52:09,999 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-03-29 15:52:1

In [ ]:
# Weather
# Weather URI
WEATHER_URI = 'https://api.openweathermap.org/data/2.5/forecast'
weather_data = requests.get(WEATHER_URI, params={
                            "units": "metric", "lat": 53.344, "lon": -6.2672, "appid": WEATHER_API_KEY})
weather_info = weather_data.json()

In [59]:
station_id = 1

# use pd.DataFrame because data is already an object
df = pd.DataFrame(weather_info['list'])
df['temperature'] = [row['main']['temp'] for row in weather_info['list']]
df['humidity'] = [row['main']['humidity'] for row in weather_info['list']]
df['wind_speed'] = [row['wind']['speed'] for row in weather_info['list']]
df['time_updated'] = pd.to_datetime(df['dt_txt'])

df['humidity'] = df['humidity'].astype('int64')

# Convert temperature and wind_speed to float64
df['temperature'] = df['temperature'].astype('float64')
df['wind_speed'] = df['wind_speed'].astype('float64')


df = df[['time_updated', 'temperature', 'wind_speed', 'humidity']]
days = ['Friday', 'Monday', 'Saturday', 'Sunday', 'Thursday', 'Tuesday',
       'Wednesday']

# One hot encode day of the week
df['weekday'] = df['time_updated'].dt.day_name()
df['hour'] = df['time_updated'].dt.hour
for day in days:
    df[day] = df['weekday'] == day

df.drop('time_updated', axis = 1, inplace=True)
df.drop('weekday', axis = 1, inplace=True)

with open(f'station_{station_id}.pkl', 'rb') as file:
    # Load the model from the file
    poly_reg_model = pickle.load(file)

poly = PolynomialFeatures(degree=3, include_bias=False)
poly_features = poly.fit_transform(df)


print(poly_reg_model.predict(poly_features))
df['predicted_available'] = poly_reg_model.predict(poly_features)


[-0.45711696 -0.93300369 -5.92561413  6.68818174 10.18708495 12.11346933
 34.73878255 15.84089644  7.98627201 10.70411377 -5.3805903   2.76787625
  7.11071606 12.15009859 18.55680163 20.40629441  5.93866236 -0.59540847
  3.00464948  9.30097485 14.75191784 19.01415029 16.38771289 15.25630441
 11.65154013  6.52551032 -2.6002261  -2.66038215  3.33197408  4.40385053
  6.74756164 10.9739237   9.81468126 10.36579422  6.25258627  0.61042918
  1.88427084 10.60411831  7.84207376  3.2991434 ]


In [60]:
df.head()

,temperature,wind_speed,humidity,hour,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,predicted_bike_stands
0,12.13,3.37,72,18,True,False,False,False,False,False,False,-0.457117
1,10.01,2.91,80,21,True,False,False,False,False,False,False,-0.933004
2,7.48,2.20,88,0,False,False,True,False,False,False,False,-5.925614
3,3.37,1.91,92,3,False,False,True,False,False,False,False,6.688182
4,3.37,2.18,91,6,False,False,True,False,False,False,False,10.187085
